In [4]:
#pip install fastdtw

In [9]:
import numpy as np
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
import pickle

In [19]:
with open("enrollment_signatures.pkl", "rb") as f:
    enrollment_signatures = pickle.load(f)

with open("verification_signatures.pkl", "rb") as f:
    verification_signatures = pickle.load(f)
print(f"Loaded {len(enrollment_signatures)} enrollment signatures")
print(f"Loaded {len(verification_signatures)} verification signatures\n")

Loaded 150 enrollment signatures
Loaded 1350 verification signatures



In [23]:
# DTW function with Sakoe-Chiba band 
def compute_dtw(sig1, sig2, radius=10):
    """
    Compute DTW distance using fastdtw with Sakoe-Chiba band radius
    sig1, sig2: np.array of shape (n_points, n_features)
    """
    distance, _ = fastdtw(sig1, sig2, dist=euclidean, radius=radius)
    return distance

# Compute DTW distances 
dtw_results = {}

print("Computing DTW distances...")

for i, (v_name, v_feat) in enumerate(verification_signatures.items(), 1):
    # Print progress every 50 signatures
    if i % 50 == 0:
        print(f"Processed {i} / {len(verification_signatures)} signatures")
    
    # Extract writer ID (assumes filename format like '001-01.tsv')
    writer_id = v_name.split('-')[0]
    
    distances = []
    # Compare with each of the 5 enrollment signatures
    for e_num in range(1, 6):
        e_name = f"{writer_id}-g-0{e_num}.tsv"
        e_feat = enrollment_signatures[e_name]
        dist = compute_dtw(v_feat, e_feat, radius=10)  # Sakoe-Chiba band
        distances.append(dist)
    
    # Use min distance as the final score
    dtw_results[v_name] = min(distances)


print("\nSample DTW distances for first 10 verification signatures:")
for k in list(dtw_results.keys())[:10]:
    print(f"{k}: {dtw_results[k]:.2f}")

# Save results
with open("dtw_results.pkl", "wb") as f:
    pickle.dump(dtw_results, f)

print("\nDTW distances saved to dtw_results.pkl")

Computing DTW distances...
Processed 50 / 1350 signatures
Processed 100 / 1350 signatures
Processed 150 / 1350 signatures
Processed 200 / 1350 signatures
Processed 250 / 1350 signatures
Processed 300 / 1350 signatures
Processed 350 / 1350 signatures
Processed 400 / 1350 signatures
Processed 450 / 1350 signatures
Processed 500 / 1350 signatures
Processed 550 / 1350 signatures
Processed 600 / 1350 signatures
Processed 650 / 1350 signatures
Processed 700 / 1350 signatures
Processed 750 / 1350 signatures
Processed 800 / 1350 signatures
Processed 850 / 1350 signatures
Processed 900 / 1350 signatures
Processed 950 / 1350 signatures
Processed 1000 / 1350 signatures
Processed 1050 / 1350 signatures
Processed 1100 / 1350 signatures
Processed 1150 / 1350 signatures
Processed 1200 / 1350 signatures
Processed 1250 / 1350 signatures
Processed 1300 / 1350 signatures
Processed 1350 / 1350 signatures

Sample DTW distances for first 10 verification signatures:
001-01.tsv: 60.95
001-02.tsv: 64.09
001-03